In [ ]:
debug=True
gm = GameManager()
gm.run_game()

In [ ]:
#######################################################################################################
#
# Classes:
# --------
# 
# Card        - implements a single game card, with functioanlities such as print and value getters.
# CardStack   - implements a list of Card objects, with functionalities such as add, remove, shuffle, 
#               initialize as empty or full.
# Player      - implements a game player. A Player has a hand (CardStack object), and can be either  
#               human or computer. 
#               the class offers several functionalities such as add and remove cards, input interface  
#               for the human user, and play functions that allow a human to play, and define the   
#               strategy of the computer.
# 
# GameManager - implements the game management. It initializes the Player objects, the deck of cards 
#               (CardStack), the table (CardStack). It's main functionalities are validation of the  
#               game rules, and the proper run cycles of the game.
#
#######################################################################################################

from random import shuffle as random_shuffle
from termcolor import colored
from IPython.display import clear_output

#######################################################################################################
# class Card:
# implements a single game card, with functioanlities such as print and value getters.
#######################################################################################################
class Card:
  
  def __init__(self, value:str):
    
    # value is a 2 char string.
    # value[0] holds the color char
    # value[1] holds the figure char

    if len(value)!= 2 or value[0] not in "gbry*" or value[1] not in "123456789+tdskc":
      raise ValueError("Card initialization error")
    
    self.__value = value
    self.__attribute = None

    if value[0] == 'g':
      self.__color = 'Green'
      self.__print_color = 'green'
    elif value[0] == 'b':
      self.__color = 'Blue'
      self.__print_color = 'blue'
    elif value[0] == 'r':
      self.__color = 'Red'
      self.__print_color = 'red'
    elif value[0] == 'y':
      self.__color = 'Yellow'
      self.__print_color = 'yellow'
    else:
      self.__color = "Special"
      self.__print_color = 'magenta'
    
    if value[1] == 't':
      self.__figure = "TAKI"
    elif value[1] == 'd':
      self.__figure = '⇄'
    elif value[1] == 's':
      self.__figure = '⛝'
    elif value[1] == 'k':
      self.__figure = '♚'
    elif value[1] == 'c':
      self.__figure = '▤'
    elif value[1] == '2':
      self.__figure = '2+'
    else:
      self.__figure = value[1]
    
  @property
  def value(self):
    return self.__value

  @property
  def color(self):
    return self.__color

  @property
  def figure(self):
    return self.__figure

  @property
  def attribute(self):
    return self.__attribute

  @attribute.setter
  def attribute(self, attribute):
    self.__attribute = attribute

  def __str__(self):
    if self.value[1] == 'c':
      if self.attribute == 'r':
        self.__print_color = 'red'
      elif self.attribute == 'g':
        self.__print_color = 'green'
      elif self.attribute == 'b':
        self.__print_color = 'blue'
      elif self.attribute == 'y':
        self.__print_color = 'yellow'
    return colored(self.__figure, self.__print_color, attrs=['bold'])

#######################################################################################################
# class CardStack:
# implements a list of Card objects, with functionalities such as add, remove, shuffle, initialize.
# the stack of cards works in FIFO. index 0 is the last card that was added to the stack.
#######################################################################################################
class CardStack:

  def __init__(self, full_stack:bool):
    
    self.__cards = []

    if full_stack:
      for color in 'gbyr':
        for i in "123456789+tds"*2:
          self.__cards.append(Card(f"{color}{i}"))
      for i in "cccc":
        self.__cards.append(Card(f"*{i}"))

  def shuffle(self):
    random_shuffle(self.__cards)

  def add(self, card:Card, sort_cards=False):
    self.__cards.insert(0,card)
    if sort_cards:
      self.__cards.sort(key=lambda card: (card.value[0],card.value[1]))
  
  def remove(self, card:Card):
    i = self.find(card)
    if i==-1:
      raise RuntimeError(f"cannot find {card.value} to remove")
    del self.__cards[i]

  def pop(self):
    return self.__cards.pop()
  
  # returns the index of the card in cards, or -1 if not found
  def find(self, card:Card):
    for index, candidate in enumerate(self.__cards):
      if candidate.value == card.value:
        return index
    return -1

  # returns a subset of the card objects in the stack.
  # index_arr can specify indexes in the stack.
  def get_cards(self, index_arr=None):
    if index_arr == None:
      return self.__cards
    ret = []
    for item in index_arr:
      ret.append(self.__cards[item])
    return ret

  def is_empty(self):
    return len(self.__cards)==0
  
  def __len__(self):
    return len(self.__cards)

  def __str__(self):
    for card in self.__cards:
      print(card, end=" ")
    return ""
  
  @property
  def card0(self):
    return self.__cards[0]

#######################################################################################################
# implements a game player. A Player has a hand (CardStack object), and can be either human or 
# computer. 
# the class offers several functionalities such as add and remove cards, input interface for the human
# user, and play functions that allow a human to play, and define the strategy of the computer.
#######################################################################################################
class Player:

  def __init__(self, name:str, computer:bool):
    
    self.__name = name
    self.__computer = computer
    self.__hand = CardStack(False)

  # gets the cards input from the user. prompts for color in case of a color change.
  def __play_manual(self, card_on_table, do_not_print_menu):
     
    new_color = "-"
    current_color = card_on_table.value[0]

    card_seq = self.get_input_for_manual_hand(do_not_print_menu)

    # user asked to draw a card from the deck
    if card_seq[0] == '0':
      return []

    # user asked to play with some of the cards
    card_objects = self.__hand.get_cards([int(i)-1 for i in card_seq])

    # user asked to change the color
    if card_objects[-1].value == '*c':
      while new_color not in 'rgby' or new_color == current_color:
        new_color = input("Change to which color (r, g, b, y)?")
        card_objects[-1].attribute = new_color

    return card_objects
    
  # returns the cards who were played by the manual user or the computer
  def play(self, card_on_table:Card, status, do_not_print_menu=False):

    new_color = "-"
    current_figure = card_on_table.value[1]

    if current_figure == 'c':
      current_color = card_on_table.attribute
    else:
      current_color = card_on_table.value[0]

    all_cards = self.__hand.get_cards()

    # has to respond with any 2+
    if status["active_2+"]:
      
      sub_cards = [card for card in all_cards if card.value[1] == '2']
      
      # can't answer the challenge
      if not sub_cards: 
        return []


    if not self.__computer:
      return self.__play_manual(card_on_table, do_not_print_menu)
    

    # --- a 2+ challenge ---- 
    if status["active_2+"]:
          
      # 2+ in the same color
      for card in sub_cards: 
        if card.value[0]==current_color: 
          return [card]
      
      return [sub_cards[0]] # 2+ in another color
    
    # play on the current color(0) or figure(1). 
    for i in range(2):

      sub_cards = [card for card in all_cards if card.value[i] == (current_color if i==0 else current_figure) and card.value[1] not in 'c']
      
      if sub_cards:

        # prioritize taki and 2+
        first = [card for card in sub_cards if card.value[1] == 't']
        last = [card for card in sub_cards if card.value[1] == '2']
        # corrected
        if len(last)>1:
          last = last[0]
        plus = [card for card in sub_cards if card.value[1] == '+']
        others = [card for card in sub_cards if card.value[1] not in 't2+']
        if not first:
          if (others or plus) and last:
            others = []
          else:
            if others:
              others= [others[0]]

        return (first + plus + others + last)

    # change color
    sub_cards = [card for card in all_cards if card.value[1] == 'c']
    if sub_cards:
      all_cards_colors = [card.value[0] for card in all_cards]
      max=-1
      for i in set(all_cards_colors):
        how_many=all_cards_colors.count(i)
        if max<how_many:
          max=how_many
          max_item = i
      if max == -1:
        new_color = 'g' # no color cards: choose arbitrarily
      else:
        new_color = max_item
      sub_cards[0].attribute = new_color
      return [sub_cards[0]]

    # nothing to play - draw a card
    return []
 
  def add_card(self, card:Card):
    self.__hand.add(card, sort_cards=True)

  def remove_card(self, card):
    self.__hand.remove(card)

  def print_hand(self):
    print(f"{self.__name}\t", end="") # has to be printed in two lines because of the colored function
    print(self.__hand)


  # prints the list of choices for the user to choose from. 
  #returns the requested indexes.
  def get_input_for_manual_hand(self, do_not_print_menu=False):
    
    requested = []
    error_text = ""
    choice_letters = [chr(97 + i) for i in range(len(self.__hand)+1)]

    while not requested:

      # get request
      print(f"{self.__name}, please enter your choice (for mutiple, use spaces as the separator):")

      if not do_not_print_menu:
        print (f"{choice_letters[0]}. Draw card")
        for index,card in enumerate(self.__hand.get_cards()):
          print(f"{choice_letters[index+1]}. {card}")
 
      requested = input().split(" ")

      if requested:
        
        if len(requested)>1 and 'a' in requested:
          error_text = "To draw a card from the deck choose 'a' only"
        elif len(requested) != len(set(requested)):
          error_text = "Duplication are not allowed!"
        else:
          wrong = [i for i in requested if i not in choice_letters]
          if wrong:
            error_text = f"There is no such choice: {wrong[0]}"
        
        if error_text:
          requested = []
          print(error_text,"\n")
          error_text=""

    
    # switch to numbers
    requested = [str(ord(i)-97) for i in requested]

    return requested

  def hand_is_empty(self): 
    return self.__hand.is_empty()
  
  @property
  def name(self):
    return self.__name

  @property
  def computer(self):
    return self.__computer

#######################################################################################################
# class GameManager:
# implements the game management. It initializes the Player objects, the deck of cards (CardStack), 
# the table (CardStack). It's main functionalities are validation of the game rules, and the proper
# run of the game.
#######################################################################################################
class GameManager:

  def __init__(self):
    
    global debug

    # initialize 1 manual player, and 3 computers
    num_of_players = 4
    name = input("What's your name?")
    self.__players = [Player("PC"+ str(i), True) for i in range(1, num_of_players)]
    self.__players.insert(0,Player(name, False))

    # initialize deck and shuffle
    self.__deck = CardStack(True)
    self.__deck.shuffle()

    # initialize the table with the first card
    self.__table = CardStack(False)
    first_card = self.__deck.pop()
  
    # reject first cards that are not adequate
    while first_card.value[1] not in "13456789":
      self.__deck.add(first_card)
      first_card = self.__deck.pop()

    self.__table.add(first_card)

    # deal the cards - push 8 cards to each player
    for p in self.__players:
      for i in range(8):
        p.add_card(self.__deck.pop())

      

    # #manip
      # if not p.computer:
      #   p.add_card(Card("g2"))
      #   p.add_card(Card("b2"))
      #   p.add_card(Card("r2"))
      #   p.add_card(Card("y2"))
    # if first_card.value[0]=='g':
    #   new_c = 'r'
    # else:
    #   new_c = 'g'

    # for p in self.__players:
    #   if p.name == 'PC1':  
    #     p.add_card(Card(new_c+'+'))
    #     p.add_card(Card(first_card.value[0]+'3'))
    #     p.add_card(Card(first_card.value[0]+'3'))
    #     p.add_card(Card(first_card.value[0]+'3'))
    #   else:
    #     for i in range(8):
    #       p.add_card(self.__deck.pop())
    #     p.add_card(Card(new_c+first_card.value[1]))
        

    self.__status = { "direction":      True, 
                      "active_2+":      0, 
                      "current_player": 0,
                      "action_played":  False
                    }

  # validates if the cards can be played on the current table.
  # returns  0 if all is ok, 
  #          1 to n - the number of cards that the player has to draw from the deck, 
  #         -1 if error
  def validate(self, cards):
    
    active_2_cash = self.__status["active_2+"]
    last_played = self.__table.card0
    current_player = self.__players[self.__status["current_player"]] 


    # USER PLAYED NOTHING

    if not cards:

        if active_2_cash:
          # user cannot reply to a 2+ challenge
          self.__status["active_2+"] = 0
          return active_2_cash

        return 1

    # USER PLAYED AT LEAST ONE CARD

    print(current_player.name, "played ", end="")
    for card in cards:
      print(card, " ", end="")
    print("\n")

    #validate: color match, or color change, or playing in the declared color of the 'Change Color' card which is on the table
    if cards[0].value[0] != last_played.value[0] and cards[0].value[1] not in 'c' and not(last_played.value[1]=='c' and last_played.attribute == cards[0].value[0]):

      #validate figure match
      if cards[0].value[1] != last_played.value[1] and cards[0].value[1] not in ('c') :

        print("figure or color mismatch")
        return -1
      
    if len(cards) == 1:
      
      if cards[0].value[1] == '+':
        return 1

      # user added 2+ and increased the cash
      if cards[0].value[1] == '2': 
        self.__status["active_2+"] += 2
        return 0

      return 0


    # USER PLAYED MORE THAN ONE CARD

    if active_2_cash:
      print("can't place more than one card during a 2+ challenge")
      return -1
    

    # scan the list. 
    # First, look for a Taki series. Then a Plus series.
    # A Taki series starts with Taki and runs as long as it’s the same color (or king or change color).
    # A Plus series starts with a plus and then adds another card or series.
    series_type = 0
    color = None
    first=True

    for index,card in enumerate(cards):
      
      if series_type == 1: # a Taki series

        if card.value[0]!=color and card.value[1]!='c':

          # run until the color has changed due to a '+' sign on top of another '+', which is the only allowed case
          if cards[index-1].value[1]=='+' and card.value[1]=='+':
            series_type = 2
            color = card.value[0]
          else:
            print("cannot place another color on a taki series: ", card.figure, " ", card.color)
            return -1
        
      
      elif series_type == 2: # a Plus series, run this (next) card only

        if card.value[1] == '+': # another plus series
          color = card.value[0]
        elif card.value[0]!=color and card.value[1]!='c':
          print("cannot place another color on a plus card")
          return -1
        elif card.value[1] == 't': # another taki series
          series_type = 1
        else:
          # reset
          series_type = 0
      
      elif card.value[1] in 't+':
        if first:
          series_type = 1 if card.value[1]=='t' else 2
          color = card.value[0]
        else:
          print("too many cards in input")
          return -1
      else:
        print("too many cards in input")
        return -1

      first = False

    # reached last card. check if there's an action
    if card.value[1] == '+':
      return 1
    if card.value[1] == '2':
      self.__status["active_2+"] += 2
      return 0
    else:
      return 0

      
  def print_table(self, player=None):
    global debug
    
    print("\n")
   
    for p in self.__players:
      if debug and (player == None or player == p):
        p.print_hand()
      elif not debug and not p.computer:
        p.print_hand()          

    print(colored("ON TABLE:", attrs=['underline']), colored(f"{self.__table.card0}", attrs=['bold']),"\n")

    
    
  # runs the cycles of the game
  def run_game(self):

    winner = False
    self.print_table()
    
    while not winner:
      
      current_player = self.__players[self.__status["current_player"]] 
      user_request_validated = -2
      self.__status["cards_played"] = False


      while user_request_validated <= -1:

        # ask current player to play, and validate the request
        player_request = current_player.play(card_on_table=self.__table.card0, status=self.__status, do_not_print_menu = False if user_request_validated==-2 else True)
        user_request_validated = self.validate(player_request)

        if current_player.computer:
          if user_request_validated == -1:
            raise ValueError("computer request cannot be validated:", [card.value for card in player_request])
 
        if user_request_validated >= 0:
          
          # place the cards one by one on the table
          for card in player_request:
            self.__table.add(card)
            current_player.remove_card(card)
          
          if player_request:
            self.__status["cards_played"] = True

          # draw cards from the deck
          for i in range(user_request_validated):
            self.add_card_from_deck(current_player)
          

      if current_player.hand_is_empty():
        # if the player's hand is empty he's the winner (unless next player has 2+, then need to continue)
        # TODO: add status winner_not_validated
        winner=True
        print(f"And the winner is........: {current_player.name}\nCONGRATULATIONS!!!")
        return

      
      #change direction
      if self.__table.card0.value[1] == 'd' and self.__status["cards_played"]:
        self.__status["direction"] = not self.__status["direction"]

      # decide the next player. if "stop" - move 2
      steps = 2 if (self.__table.card0.value[1] == 's' and self.__status["cards_played"]) else 1
      self.__status["current_player"] = (self.__status["current_player"] + (steps if self.__status["direction"] else -steps)) % len(self.__players)

      input("...")
      clear_output(True)

      # show table
      self.print_table(player = self.__players[self.__status["current_player"]])

      input("...")

  def add_card_from_deck(self, player:Player):
    
    global debug

    if self.__deck.is_empty():
      self.recharge_deck()
    
    card = self.__deck.pop()
    player.add_card(card)

    message = f"{player.name} drawn card" + f": {card}" if (not player.computer or debug) else ""

    print(message)

  # recharges the deck when it is empty
  def recharge_deck(self):

    # keep what's on table (so the game can continue)
    temp = self.__table.get_cards()
    on_table = temp.pop(0) 

    # deck becomes the table without the first card
    temp = self.__deck
    self.__deck = self.__table
    self.__deck.shuffle()

    # return the play card to the table
    self.__table = CardStack(False)
    self.__table.add(on_table)

  @property
  def status(self):
    return self.__status




In [ ]:
# Known bugs:
# when computer has several colors of 2+ which are different than the 2+ on the table, it attemts to put all the 2+
# when computer changes color and it has two cards - one of each color (eg: red, yellow), the color is not determined (no color)
# game should not end when 2+ is the last card (add status winner_not_validated?)
# on a 2 players game, when manual user puts ⇄  it loses its turn
# on a 2 players game, when manual user puts ⇄ as the last card, it is announced as the winner (is it a bug?)

In [ ]:
# QA

debug=True
gm = GameManager()




def qa_set_card0(self, card:Card):
  self.__table.add(card)
  print("card0:", self.__table.card0)

def qa_reset_active_2(self):
  self.__status["active_2+"] = 0


gm.qa_set_card0(Card('y1'))
print("test 10")
test_cards = []
if gm.validate(test_cards)!=1:
  print("*** err ***")

print("test 20")
test_cards = [Card('g1')]
if gm.validate(test_cards)!=0:
  print("*** err ***")

print("test 21")
test_cards = [Card('g3')]
if gm.validate(test_cards)!=-1:
  print("*** err ***")

print("test 22")
test_cards = [Card('y3')]
if gm.validate(test_cards)!=0:
  print("*** err ***")

print("test 23")
test_cards = [Card('y2')]
if gm.validate(test_cards)!=0:
  print("*** err ***")

gm.qa_reset_active_2()

print("test 24")
test_cards = [Card('g2')]
if gm.validate(test_cards)!=-1:
  print("*** err ***")

gm.qa_reset_active_2()


print("test 30")
test_cards = [Card('g1'), Card('g3')]
if gm.validate(test_cards)!=-1:
  print("*** err ***")

print("test 40")
test_cards = [Card('g1'), Card('*c')]
if gm.validate(test_cards)!=-1:
  print("*** err ***")

print("test 50")
test_cards = [Card('g1'), Card('*c')]
if gm.validate(test_cards)!=-1:
  print("*** err ***")

print("test 51")
test_cards = [Card('g+'), Card('g6')]
if gm.validate(test_cards)!=-1:
  print("*** err ***")

print("test 52")
test_cards = [Card('y+'), Card('g6')]
if gm.validate(test_cards)!=-1:
  print("*** err ***")

print("test 53")
test_cards = [Card('y+'), Card('y6')]
if gm.validate(test_cards)!=0:
  print("*** err ***")

print("test 54")
test_cards = [Card('y6'), Card('y+')]
if gm.validate(test_cards)!=-1:
  print("*** err ***")

print("test 55")
test_cards = [Card('y+')]
if gm.validate(test_cards)!=1:
  print("*** err ***")

print("test 60")
test_cards = [Card('y+'), Card('y6'), Card('y7')]
if gm.validate(test_cards)!=-1:
  print("*** err ***")

print("test 61")
test_cards = [Card('y+'), Card('y+'), Card('y7')]
if gm.validate(test_cards)!=0:
  print("*** err ***")

print("test 62")
test_cards = [Card('y+'), Card('y+'), Card('y+')]
if gm.validate(test_cards)!=1:
  print("*** err ***")

print("test 63")
test_cards = [Card('y+'), Card('g+')]
if gm.validate(test_cards)!=1:
  print("*** err ***", gm.validate(test_cards))

print("test 64")
test_cards = [Card('y+'), Card('g+'), Card('g7')]
if gm.validate(test_cards)!=0:
  print("*** err ***")

print("test 70")
test_cards = [Card('gt')]
if gm.validate(test_cards)!=-1:
  print("*** err ***")

print("test 71")
test_cards = [Card('yt')]
if gm.validate(test_cards)!=0:
  print("*** err ***")

print("test 72")
test_cards = [Card('yt'),Card('y8'), Card('y9'), Card('y6')]
if gm.validate(test_cards)!=0:
  print("*** err ***")

print("test 73")
test_cards = [Card('yt'),Card('y+'), Card('y9'), Card('y6')]
if gm.validate(test_cards)!=0:
  print("*** err ***")

print("test 74")
test_cards = [Card('yt'),Card('y+'), Card('y9'), Card('y+')]
if gm.validate(test_cards)!=1:
  print("*** err ***")

print("test 75")
test_cards = [Card('y+'),Card('yt'), Card('y9'), Card('y6')]
if gm.validate(test_cards)!=0:
  print("*** err ***")

print("test 80")
test_cards = [Card('yt'),Card('y+'), Card('y9'), Card('y+'), Card('r+'), Card('r8')]
if gm.validate(test_cards)!=0:
  print("*** err ***")

print("test 81")
test_cards = [Card('yt'),Card('y+'), Card('y9'), Card('y+'), Card('r+'), Card('r8'), Card('r+')]
if gm.validate(test_cards)!=1:
  print("*** err ***")

print("test 82")
test_cards = [Card('yt'),Card('y+'), Card('y9'), Card('y+'), Card('r+'), Card('rt'), Card('r9'), Card('r5'), Card('r4')]
if gm.validate(test_cards)!=0:
  print("*** err ***")